# Analytics

#### Date: 2020/02

#### SUMMARY:

- This notebook represents the project quality analysis of the date exposed right above. 

### TEAM:

##### Semester: YYYY/0X
##### Professor: Hilmer Neri

##### Members:

- Member x
- Member y

### LIBRARIES

In [2]:
# Deal with data
import pandas as pd
import numpy as np
import json
from glob import glob
import os

# Deal with API request
import urllib3
from urllib3 import request

# Deal with visualization
import seaborn as sns
import matplotlib.pyplot as plt

### GRAPH SETTINGS

In [3]:
%config InlineBackend.figure_format ='retina'
sns.set(font_scale=1.5)
sns.set_style('darkgrid',
              {'xtick.bottom' : True,
               'ytick.left': True,
               'grid.linestyle':'--',
               'font.monospace': ['Computer Modern Typewriter'],
               'axes.edgecolor' : 'white'})

### DATAFRAME SETTINGS

In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

### SonarCloud

In [5]:
repos = [
    '2021.1-Cartografia-social-api-comunidades',
    '2021.1-Cartografia-social-api-gateway',
    '2021.1-Cartografia-social-api-mapas',
    '2021.1-Cartografia-social-api-midia',
    '2021.1-Cartografia-social-api-users',
    '2021.1-Cartografia-social-front'
]

language = [['2021.1-Cartografia-social-api-comunidades', 'ts'], 
            ['2021.1-Cartografia-social-api-gateway', 'ts'], 
            ['2021.1-Cartografia-social-api-mapas', 'ts'], 
            ['2021.1-Cartografia-social-api-midia', 'ts'], 
            ['2021.1-Cartografia-social-api-users', 'ts'], 
            ['2021.1-Cartografia-social-front', 'js']]

repos_language = {}

for item in language:
    repos_language[f"{item[0]}"] = item[1]

In [6]:
repos_language

{'2021.1-Cartografia-social-api-comunidades': 'ts',
 '2021.1-Cartografia-social-api-gateway': 'ts',
 '2021.1-Cartografia-social-api-mapas': 'ts',
 '2021.1-Cartografia-social-api-midia': 'ts',
 '2021.1-Cartografia-social-api-users': 'ts',
 '2021.1-Cartografia-social-front': 'js'}

##### Path to the folder with all your jsons

In [7]:
jsons = glob('**/*.json') # add the path here

In [8]:
def read_json(json_path):
    
    with open(json_path) as json_file:
        json_obj = json.load(json_file)
        
    return json_obj

def create_base_component_df(json_list):
    
    df = pd.DataFrame()

    for i in json_list:

        base_component = read_json(i)

        base_component_data = base_component['baseComponent']['measures']

        base_component_df = pd.DataFrame(base_component_data)

        base_component_df['filename'] = os.path.basename(i)

        df = df.append(base_component_df, ignore_index=True)
        
    aux_df = df['filename'].str.split(r"fga-eps-mds-2021_1-(.*?)-(.*?)_(.*?).json", expand=True)
    
    df['repository'] = aux_df[1]
    
    df['version'] = aux_df[2]
    
    df = df.sort_values(by=['repository', 'version'])
        
    return df

#### Create base component dataframe

In [9]:
base_component_df = create_base_component_df(jsons)

In [10]:
base_component_df.head(10)

,metric,value,bestValue,filename,repository,version
42,duplicated_lines_density,0.0,True,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021
43,functions,31,NaN,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021
44,security_rating,1.0,True,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021
45,files,12,NaN,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021
46,complexity,36,NaN,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021
47,ncloc,284,NaN,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021
48,coverage,85.9,False,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021
49,reliability_rating,1.0,True,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021
50,comment_lines_density,0.7,False,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021
51,duplicated_lines_density,0.0,True,fga-eps-mds-2021_1-Cartografia-social-api-comu...,Cartografia,social-api-comunidades-10-15-2021


#### Create dataframe per file

In [11]:
metric_list = ['files',
               'functions',
               'complexity',
               'comment_lines_density',
               'duplicated_lines_density',
               'coverage',
               'ncloc',
               'security_rating',
               'tests',
               'test_success_density',
               'test_execution_time',
               'reliability_rating']

len(metric_list)

12

In [12]:
def metric_per_file(json):
    
    file_json = []
    
    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)
            
    return file_json

def generate_file_dataframe_per_release(metric_list, json, language_extension):
    
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)
    
    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass
        
    df.reset_index(inplace = True)
    df = df.rename({'index': 'path'}, axis=1).drop(['files'], axis=1)

    return df

def create_file_df(json_list):
    
    df = pd.DataFrame()

    for i in json_list:

        file_component = read_json(i)
        
        file_component_data = metric_per_file(file_component)

        file_component_df = generate_file_dataframe_per_release(metric_list, file_component_data, language_extension = 'js')

        file_component_df['filename'] = os.path.basename(i)

        df = df.append(file_component_df, ignore_index=True)
        
    # replace TeamName by yours.    
    aux_df = df['filename'].str.split(r"fga-eps-mds-2021_1-(.*?)-(.*?)_(.*?).json", expand=True)
    
    df['repository'] = aux_df[1]
    
    df['version'] = aux_df[2]
    
    df = df.sort_values(by=['repository', 'version'])
        
    return df

In [13]:
file_component_df = create_file_df(jsons)

In [14]:
file_component_df.head(10)

,path,functions,complexity,comment_lines_density,duplicated_lines_density,coverage,ncloc,security_rating,tests,test_success_density,test_execution_time,reliability_rating,filename,repository,version
297,src/store/actions/actionTypes.js,0,0,0.0,0.0,0.0,5,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30
298,src/services/api.js,1,1,0.0,0.0,0.0,14,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30
299,src/services/api2.js,1,1,0.0,0.0,0.0,14,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30
300,src/app/App.js,1,1,0.0,0.0,0.0,8,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30
301,src/store/actions/auth.js,2,2,0.0,0.0,0.0,12,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30
302,src/store/reducers/auth.js,1,4,0.0,0.0,0.0,17,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30
303,src/store/selectors/auth.js,1,1,0.0,0.0,NaN,1,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30
304,src/components/Fabs/index.js,6,9,0.0,0.0,0.0,55,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30
305,src/components/CreatePoint/index.js,16,23,0.0,0.0,0.0,157,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30
306,src/components/MarkerDetails/index.js,3,6,14.5,0.0,0.0,53,1.0,NaN,100.0,NaN,1.0,fga-eps-mds-2021_1-Cartografia-social-front-09...,Cartografia,social-front-09-14-2021-23-30


#### Create dataframe per repository

In [15]:
repository_dataframes = {} 

for repository in repos:
    df = file_component_df[file_component_df['repository'] == f"{repository}"]
    df.name = f"{repository}"
    repository_dataframes[f"{repository}"] = df
    
repository_dataframes.keys()

dict_keys(['2021.1-Cartografia-social-api-comunidades', '2021.1-Cartografia-social-api-gateway', '2021.1-Cartografia-social-api-mapas', '2021.1-Cartografia-social-api-midia', '2021.1-Cartografia-social-api-users', '2021.1-Cartografia-social-front'])

In [ ]:
api_comunidades_df = file_component_df[file_component_df['repository'] == '2021.1-Cartografia-social-api-comunidades']
api_gateway_df = file_component_df[file_component_df['repository'] == '2021.1-Cartografia-social-api-gateway']
api_mapas_df = file_component_df[file_component_df['repository'] == '2021.1-Cartografia-social-api-mapas']
api_midia_df = file_component_df[file_component_df['repository'] == '2021.1-Cartografia-social-api-midia']
api_users_df = file_component_df[file_component_df['repository'] == '2021.1-Cartografia-social-api-users']
front_df = file_component_df[file_component_df['repository'] == '2021.1-Cartografia-social-api-front']

In [16]:
def _nloc(df):
    ncloc = 0
    for each in df['ncloc']:
        ncloc += int(each)

    return ncloc

### Measure calculations according Q-Rapids quality model

# Quality Aspect - Maintainability
## Factor - Code Quality

##### COMPLEXITY

In [17]:
def m1(df):
    
    density_non_complex_files = len(df[(df['complexity'].astype(float)/df['functions'].astype(float)) < 10])/len(df)
    
    return density_non_complex_files

##### COMMENTS

In [18]:
def m2(df):
    
    density_comment_files = len(df[(df['comment_lines_density'].astype(float) > 10) & (df['comment_lines_density'].astype(float) < 30)])/len(df)
    
    return density_comment_files

##### DUPLICATIONS

In [19]:
def m3(df):
    
    duplication = len(df[(df['duplicated_lines_density'].astype(float) < 5)])/len(df)
    
    return duplication

# Quality Aspect - Reliability
## Factor - Testing Status

##### Passed tests

In [20]:
def m4(df):

    passed_tests = df['test_success_density'].astype(float).median() / 100

    return passed_tests

##### Fast test builds

In [21]:
def m5(df):

    density_fast_test_builds = len(df[(df['test_execution_time'].astype(float)) < 300]) / sum(df['tests'].astype(float))

    return density_fast_test_builds

##### Test coverage

In [22]:
def m6(df):

    density_test_coverage = len(df[(df['coverage'].astype(float) > 60)]) / len(df)

    return density_test_coverage

### Calculate m1, m2, m3, m4, m5 and m6 for each repository

In [23]:
def create_metrics_df(df):
    
    version_vec = df['version'].unique()
    
    m1_list = []
    m2_list = []
    m3_list = []
    m4_list = []
    m5_list = []
    m6_list = []

    ncloc_list = []
    repository_list = []
    version_list = []
    
    metrics_df = pd.DataFrame()
    
    for version in version_vec:

        version_df = df[df['version'] == version]

        m1_list.append(m1(version_df))
        m2_list.append(m2(version_df))
        m3_list.append(m3(version_df))
        m4_list.append(m3(version_df))
        m5_list.append(m3(version_df))
        m6_list.append(m3(version_df))

        ncloc_list.append(_ncloc(version_df))
        repository_list.append(version_df['repository'].iloc[0])
        version_list.append(version)
        
    metrics_df = pd.DataFrame({'m1': m1_list,
                               'm2': m2_list,
                               'm3': m3_list,
                               'm4': m3_list,
                               'm5': m3_list,
                               'm6': m3_list,
                               'repository': repository_list, 
                               'version': version_list,
                               'ncloc': ncloc_list})
        
    return metrics_df

In [27]:
repository_metrics = {}

for repository, repo_df in repository_dataframes.items():   
    metrics_df = create_metrics_df(repo_df)
    metrics_df.name = f"{repository}"
    repository_metrics[f"{repository}"] = metrics_df
    
repository_dataframes.keys()

dict_keys(['2021.1-Cartografia-social-api-comunidades', '2021.1-Cartografia-social-api-gateway', '2021.1-Cartografia-social-api-mapas', '2021.1-Cartografia-social-api-midia', '2021.1-Cartografia-social-api-users', '2021.1-Cartografia-social-front'])

In [ ]:
comunidades_metrics = create_metrics_df(eccoar_gateway_df)
eccoar_complaint_metrics = create_metrics_df(eccoar_complaint_df)
eccoar_frontend_metrics = create_metrics_df(eccoar_frontend_df)
eccoar_mailer_metrics = create_metrics_df(eccoar_mailer_df)
eccoar_reports_metrics = create_metrics_df(eccoar_reports_df)
eccoar_users_metrics = create_metrics_df(eccoar_users_df)

### Data visualization

- You must do this for each of your repositories

In [39]:
fig = plt.figure(figsize=(20, 10))

plt.plot(repo1['m1'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m2'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m3'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m4'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m5'], linewidth=3, marker='o', markersize=10)
plt.plot(repo1['m6'], linewidth=3, marker='o', markersize=10)

NameError: name 'repo1' is not defined

<Figure size 1440x720 with 0 Axes>

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.plot(repo2['m1'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m2'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m3'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m4'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m5'], linewidth=3, marker='o', markersize=10)
plt.plot(repo2['m6'], linewidth=3, marker='o', markersize=10)

...

### Sub characteristic aggregation

- You must do this for each of your repositories

In [ ]:
psc1, psc2 = 1
pc1, pc2 = 0.5
pm1 = 0.33
pm2 = 0.33
pm3 = 0.33
pm4 = 0.15
pm5 = 0.15
pm6 = 0.7

repo1['code_quality'] = ((repo1['m1']*pm1) + (repo1['m2']*pm2) + (repo1['m3']*pm3)) * psc1
repo2['code_quality'] = ((repo2['m1']*pm1) + (repo2['m2']*pm2) + (repo2['m3']*pm3)) * psc1
...

repo1['testing_status'] = ((repo1['m4']*pm4) + (repo1['m5']*pm5) + (repo1['m6']*pm6)) * psc2
repo2['testing_status'] = ((repo2['m4']*pm4) + (repo2['m5']*pm5) + (repo2['m6']*pm6)) * psc2
...

In [ ]:
fig = plt.figure(figsize=(20, 10))


plt.plot(repo1['code_quality'], linewidth=3, marker='o', markersize=5)
plt.plot(repo2['code_quality'], linewidth=3, marker='o', markersize=5)
...

In [ ]:
fig = plt.figure(figsize=(20, 10))


plt.plot(repo1['testing_status'], linewidth=3, marker='o', markersize=5)
plt.plot(repo2['testing_status'], linewidth=3, marker='o', markersize=5)
...


In [ ]:
metrics_df = pd.concat([repo1_metrics, repo2_metrics, ...], ignore_index=True)

metrics_df['maintainability'] = metrics_df['code_quality'] * pc1
metrics_df['Reliability'] = metrics_df['testing_status'] * pc2
metrics_df['total'] = metrics_df['maintainability'] + metrics_df['Reliability']

In [ ]:
metrics_df

In [ ]:
fig = plt.figure(figsize=(20, 10))


plt.plot(metrics_df['maintainability'], linewidth=3, marker='o', markersize=5)
plt.plot(metrics_df['Reliability'], linewidth=3, marker='o', markersize=5)


plt.ylim(.45,.7)

In [ ]:
fig = plt.figure(figsize=(20, 10))


plt.plot(metrics_df['total'], linewidth=3, marker='o', markersize=5)


plt.ylim(.45,.7)


In [ ]:
# ANALYSIS_DATE FORMAT: MM-DD-YYYY

metrics_df.to_excel('data/ga-eps-mds-2021_1-NOME_PROJETO-DATASET-ANALYSIS_DATE.xlsx', index = False)
metrics_df.to_csv('data/ga-eps-mds-2021_1-NOME_PROJETO-DATASET-ANALYSIS_DATE.csv', index = False)